# get_paths_labels

Generate the paths and labels for training set, validation set and test set.


In [1]:
import os    
import numpy as np
import pickle
import re 

import numpy as np
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms

from PIL import Image

# 1. get_paths_labels for training set train06-train25 and val set train01-train05

In [2]:
def get_frame_dirs(root_dir):
    file_paths = []
    file_names = []
    for lists in os.listdir(root_dir):
        path = os.path.join(root_dir, lists)
        if os.path.isdir(path):
            file_paths.append(path)
            file_names.append(os.path.basename(path))
    file_names.sort(key=lambda x:int(x))
    file_paths.sort(key=lambda x:int(os.path.basename(x)))
    return file_names, file_paths


def get_step_dirs(root_dir):
    file_paths = []
    file_names = []
    for lists in os.listdir(root_dir):
        path = os.path.join(root_dir, lists)
        if not os.path.isdir(path):
            file_paths.append(path)
            file_names.append(os.path.basename(path))
    file_names.sort()
    file_paths.sort()
    return file_names, file_paths

In [61]:
root_dir2 = '/media/yilin/datasets/cataracts/'
img_dir2 = os.path.join(root_dir2, 'frames') 
phase_dir2 = os.path.join(root_dir2, 'gt')    

In [6]:
img_dir_names2, img_dir_paths2 = get_frame_dirs(img_dir2)
phase_file_names2, phase_file_paths2 = get_step_dirs(phase_dir2)

In [6]:
img_dir_names2

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50']

In [7]:
img_dir_paths2

['/media/yilin/datasets/cataracts/frames/1',
 '/media/yilin/datasets/cataracts/frames/2',
 '/media/yilin/datasets/cataracts/frames/3',
 '/media/yilin/datasets/cataracts/frames/4',
 '/media/yilin/datasets/cataracts/frames/5',
 '/media/yilin/datasets/cataracts/frames/6',
 '/media/yilin/datasets/cataracts/frames/7',
 '/media/yilin/datasets/cataracts/frames/8',
 '/media/yilin/datasets/cataracts/frames/9',
 '/media/yilin/datasets/cataracts/frames/10',
 '/media/yilin/datasets/cataracts/frames/11',
 '/media/yilin/datasets/cataracts/frames/12',
 '/media/yilin/datasets/cataracts/frames/13',
 '/media/yilin/datasets/cataracts/frames/14',
 '/media/yilin/datasets/cataracts/frames/15',
 '/media/yilin/datasets/cataracts/frames/16',
 '/media/yilin/datasets/cataracts/frames/17',
 '/media/yilin/datasets/cataracts/frames/18',
 '/media/yilin/datasets/cataracts/frames/19',
 '/media/yilin/datasets/cataracts/frames/20',
 '/media/yilin/datasets/cataracts/frames/21',
 '/media/yilin/datasets/cataracts/frames/22

In [10]:
phase_file_names2

['train01.csv',
 'train02.csv',
 'train03.csv',
 'train04.csv',
 'train05.csv',
 'train06.csv',
 'train07.csv',
 'train08.csv',
 'train09.csv',
 'train10.csv',
 'train11.csv',
 'train12.csv',
 'train13.csv',
 'train14.csv',
 'train15.csv',
 'train16.csv',
 'train17.csv',
 'train18.csv',
 'train19.csv',
 'train20.csv',
 'train21.csv',
 'train22.csv',
 'train23.csv',
 'train24.csv',
 'train25.csv']

In [9]:
phase_file_paths2

['/media/yilin/datasets/cataracts/gt/train01.csv',
 '/media/yilin/datasets/cataracts/gt/train02.csv',
 '/media/yilin/datasets/cataracts/gt/train03.csv',
 '/media/yilin/datasets/cataracts/gt/train04.csv',
 '/media/yilin/datasets/cataracts/gt/train05.csv',
 '/media/yilin/datasets/cataracts/gt/train06.csv',
 '/media/yilin/datasets/cataracts/gt/train07.csv',
 '/media/yilin/datasets/cataracts/gt/train08.csv',
 '/media/yilin/datasets/cataracts/gt/train09.csv',
 '/media/yilin/datasets/cataracts/gt/train10.csv',
 '/media/yilin/datasets/cataracts/gt/train11.csv',
 '/media/yilin/datasets/cataracts/gt/train12.csv',
 '/media/yilin/datasets/cataracts/gt/train13.csv',
 '/media/yilin/datasets/cataracts/gt/train14.csv',
 '/media/yilin/datasets/cataracts/gt/train15.csv',
 '/media/yilin/datasets/cataracts/gt/train16.csv',
 '/media/yilin/datasets/cataracts/gt/train17.csv',
 '/media/yilin/datasets/cataracts/gt/train18.csv',
 '/media/yilin/datasets/cataracts/gt/train19.csv',
 '/media/yilin/datasets/catarac

In [3]:
# downsample_rate = 30  30fps -> 1fps
def gen_paths_labels_train(root_dir2,img_dir2,phase_dir2,downsample_rate):
    # get dir of frame, dir of label
    img_dir_names2, img_dir_paths2 = get_frame_dirs(img_dir2)
    phase_file_names2, phase_file_paths2 = get_step_dirs(phase_dir2)
    
    paths_labels_info = []
    for i in range(len(phase_file_names2)):
        phase_file = open(phase_file_paths2[i])
        
        info_ = []
        first_line = True
        for phase_line in phase_file:
            phase_split = phase_line.split()
            if first_line:
                first_line = False
                continue
                
            frame_no = re.findall(r'\d+', phase_split[0])  # yilin
            
            # the processed frames of the video start from frame 0
            # gt csv file start from frame 1
            if (int(frame_no[0])) % downsample_rate == 0: # int(phase_split[0])
                info_each = []
                # the processed frames of the video start from frame 0
                frame_name = str(int(frame_no[0])-1)
                img_file_each_path = os.path.join(img_dir_paths2[i], frame_name + '.jpg') #  phase_split[0] frame_no[0]
                info_each.append(img_file_each_path)
                info_each.append(frame_no[1]) 
                info_.append(info_each)    
                
        paths_labels_info.append(info_)
        
    with open('./cataracts_train.pkl', 'wb') as f:
        pickle.dump(paths_labels_info, f)
        

    return paths_labels_info 

In [7]:
paths_labels_info = gen_paths_labels_train(root_dir2,img_dir2,phase_dir2,10)

In [11]:
paths_labels_info

[[['/media/yilin/datasets/cataracts/frames/1/9.jpg', '0'],
  ['/media/yilin/datasets/cataracts/frames/1/19.jpg', '0'],
  ['/media/yilin/datasets/cataracts/frames/1/29.jpg', '0'],
  ['/media/yilin/datasets/cataracts/frames/1/39.jpg', '0'],
  ['/media/yilin/datasets/cataracts/frames/1/49.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/59.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/69.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/79.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/89.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/99.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/109.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/119.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/129.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/139.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/149.jpg', '3'],
  ['/media/yilin/datasets/cataracts/frames/1/159.jpg', '3'],
  ['/media/yilin/datasets/cataracts

In [20]:
paths_labels_info[0][1437]

['/media/yilin/datasets/cataracts/frames/1/14379.jpg', '0']

In [4]:
def get_paths_labels_train(paths_labels_info):
    
    # paths of all training/val videos
    train_file_paths_50 = []
    val_file_paths_50 = []

    # labels of all training/val videos
    train_labels_50 = []
    val_labels_50 = []

    # the number of frames of each video 
    train_num_each_50 = []
    val_num_each_50 = []
    
    #train06-train25 for training set
    for i in range(5,25): 
        train_num_each_50.append(len(paths_labels_info[i]))
        for j in range(len(paths_labels_info[i])):
            train_file_paths_50.append(paths_labels_info[i][j][0])
            train_labels_50.append(paths_labels_info[i][j][1:])


    print(len(train_file_paths_50))
    print(len(train_labels_50))

     
    #  train01- train05 for validation set
    for i in range(5): 
        val_num_each_50.append(len(paths_labels_info[i]))
        for j in range(len(paths_labels_info[i])):
            val_file_paths_50.append(paths_labels_info[i][j][0])
            val_labels_50.append(paths_labels_info[i][j][1:])
            

    print(len(val_file_paths_50))
    print(len(val_labels_50))
    
    
    train_val_paths_labels = []
    train_val_paths_labels.append(train_file_paths_50) 
    train_val_paths_labels.append(val_file_paths_50)

    train_val_paths_labels.append(train_labels_50)
    train_val_paths_labels.append(val_labels_50)

    train_val_paths_labels.append(train_num_each_50)
    train_val_paths_labels.append(val_num_each_50)
    
    with open('train_val_paths_labels.pkl', 'wb') as f:
        pickle.dump(train_val_paths_labels, f)
    
    return train_val_paths_labels

In [5]:
root_dir2 = '/media/yilin/datasets/cataracts/'
img_dir2 = os.path.join(root_dir2, 'frames') 
phase_dir2 = os.path.join(root_dir2, 'gt') #  gt1-25 train 26-50 test

# sample rate 30 meaning 30fps -> 1 fps
sample_rate = 30
paths_labels_info = gen_paths_labels_train(root_dir2,img_dir2,phase_dir2,sample_rate)

In [6]:
# get paths and labels
train_val_paths_labels = get_paths_labels_train(paths_labels_info)

14160
14160
2323
2323


In [101]:
train_val_paths_labels[0]

['/media/yilin/datasets/cataracts/frames/6/29.jpg',
 '/media/yilin/datasets/cataracts/frames/6/59.jpg',
 '/media/yilin/datasets/cataracts/frames/6/89.jpg',
 '/media/yilin/datasets/cataracts/frames/6/119.jpg',
 '/media/yilin/datasets/cataracts/frames/6/149.jpg',
 '/media/yilin/datasets/cataracts/frames/6/179.jpg',
 '/media/yilin/datasets/cataracts/frames/6/209.jpg',
 '/media/yilin/datasets/cataracts/frames/6/239.jpg',
 '/media/yilin/datasets/cataracts/frames/6/269.jpg',
 '/media/yilin/datasets/cataracts/frames/6/299.jpg',
 '/media/yilin/datasets/cataracts/frames/6/329.jpg',
 '/media/yilin/datasets/cataracts/frames/6/359.jpg',
 '/media/yilin/datasets/cataracts/frames/6/389.jpg',
 '/media/yilin/datasets/cataracts/frames/6/419.jpg',
 '/media/yilin/datasets/cataracts/frames/6/449.jpg',
 '/media/yilin/datasets/cataracts/frames/6/479.jpg',
 '/media/yilin/datasets/cataracts/frames/6/509.jpg',
 '/media/yilin/datasets/cataracts/frames/6/539.jpg',
 '/media/yilin/datasets/cataracts/frames/6/569.jp

# 2. get_paths_labels_test : test01-test25

challenge2020 dev: test01,test07,test14,test16,test19

In [7]:
# downsample_rate = 30
def gen_paths_labels_test(root_dir2,img_dir2,phase_dir2,downsample_rate):
    # get dir of frame, dir of label
    img_dir_names2, img_dir_paths2 = get_frame_dirs(img_dir2)
    phase_file_names2, phase_file_paths2 = get_step_dirs(phase_dir2)
    
    paths_labels_info = []
    for i in range(len(phase_file_names2)):
        phase_file = open(phase_file_paths2[i])
        
        info_ = []
        first_line = True
        for phase_line in phase_file:
            phase_split = phase_line.split()
            if first_line:
                first_line = False
                continue
                
            frame_no = re.findall(r'\d+', phase_split[0])  # yilin
            
            # the processed frames of the video start from frame 0
            # gt csv file start from frame 1
            if (int(frame_no[0])) % downsample_rate == 0: # int(phase_split[0])
                info_each = []
                # the processed frames of the video start from frame 0
                frame_name = str(int(frame_no[0])-1)
                img_file_each_path = os.path.join(img_dir_paths2[i], frame_name + '.jpg') #  phase_split[0] frame_no[0]
                info_each.append(img_file_each_path)
                info_each.append(frame_no[1]) 
                info_.append(info_each)    
                
        paths_labels_info.append(info_)
        
    with open('./cataracts_test.pkl', 'wb') as f:
        pickle.dump(paths_labels_info, f)
        

    return paths_labels_info 

In [8]:
def get_paths_labels_test(num_video_test,paths_labels_info):
    
    # paths of all training/val videos
    test_file_paths_50 = []


    # labels of all training/val videos
    test_labels_50 = []


    # the number of frames of each video 
    test_num_each_50 = []

    # test videos 25
    for i in range(num_video_test): 
        test_num_each_50.append(len(paths_labels_info[i]))
        for j in range(len(paths_labels_info[i])):
            test_file_paths_50.append(paths_labels_info[i][j][0])
            test_labels_50.append(paths_labels_info[i][j][1:])
            
    
    

    print(len(test_file_paths_50))
    print(len(test_labels_50))

    
    
    test_paths_labels = []
    test_paths_labels.append(test_file_paths_50)

    test_paths_labels.append(test_labels_50)

    test_paths_labels.append(test_num_each_50)

    
    with open('test_paths_labels.pkl', 'wb') as f:
        pickle.dump(test_paths_labels, f)
    
    return test_paths_labels

In [9]:
root_dir2 = '/media/yilin/datasets/cataracts/'
img_dir2 = os.path.join(root_dir2, 'test_frames') 
phase_dir2 = os.path.join(root_dir2, 'test_gt') #  gt1-25 train 26-50 test


# sample rate 30 meaning 30fps -> 1 fps
sample_rate = 30
paths_labels_info = gen_paths_labels_test(root_dir2,img_dir2,phase_dir2,sample_rate)

# test video 1-25 for test
num_video_test = 25
test_paths_labels = get_paths_labels_test(num_video_test,paths_labels_info)

15425
15425


In [10]:
len(test_paths_labels[2])

25

# 3. compute weights of weighted cross entropy loss

In [11]:
import torch

In [15]:
def compute_weights_WCE(num_each_class):
    weights = torch.tensor(num_each_class, dtype=torch.float32)
    weights = 1.0/(weights / weights.sum())
    # weights = 1.0 / weights
    weights = weights / weights.sum()

    print(weights)
    return weights

In [61]:
# num_each_class = [900,15000,800]
# weight = compute_weights_WCE(num_each_class)

In [62]:
root_dir2 = '/media/yilin/datasets/cataracts/'
img_dir2 = os.path.join(root_dir2, 'frames') 
phase_dir2 = os.path.join(root_dir2, 'gt') #  gt1-25 train 26-50 test

# sample rate 30 meaning 30fps -> 1 fps
sample_rate = 30
paths_labels_info = gen_paths_labels_train(root_dir2,img_dir2,phase_dir2,sample_rate)

In [53]:
num_step_0 = 0
num_step_1 = 0
num_step_2 = 0
num_step_3 = 0
num_step_4 = 0
num_step_5 = 0
num_step_6 = 0
num_step_7 = 0
num_step_8 = 0
num_step_9 = 0
num_step_10 = 0
num_step_11 = 0
num_step_12 = 0
num_step_13 = 0
num_step_14 = 0
num_step_15 = 0
num_step_16 = 0
num_step_17 = 0
num_step_18 = 0

In [ ]:
# LENGTH labels of video6-25

461,625,915,532,852,
475,719,467,684,458,
607,414,465,2368,564,
515,688,372,619,1360
 

In [54]:
# add from video6-25 index 5-24
video_no = 24
length_labels = 1360

for i in range(length_labels):
    gt_step = paths_labels_info[video_no][i][1]
    if gt_step == '0':
        num_step_0 += 1
        
    elif gt_step == '1':
        num_step_1 += 1
    elif gt_step == '2':
        num_step_2 += 1
    elif gt_step == '3':
        num_step_3 += 1
    elif gt_step == '4':
        num_step_4 += 1
    elif gt_step == '5':
        num_step_5 += 1
        
    elif gt_step == '6':
        num_step_6 += 1
    elif gt_step == '7':
        num_step_7 += 1
    elif gt_step == '8':
        num_step_8 += 1
    elif gt_step == '9':
        num_step_9 += 1
    elif gt_step == '10':
        num_step_10 += 1
        
    elif gt_step == '11':
        num_step_11 += 1
    elif gt_step == '12':
        num_step_12 += 1
    elif gt_step == '13':
        num_step_13 += 1
    elif gt_step =='14':
        num_step_14 += 1
    elif gt_step == '15':
        num_step_15 += 1
        
    elif gt_step == '16':
        num_step_16 += 1
    elif gt_step == '17':
        num_step_17 += 1
    elif gt_step == '18':
        num_step_18 += 1    

In [ ]:
# all video 6-25 
print(num_step_0,num_step_1,num_step_2,num_step_3,num_step_4,num_step_5,num_step_6,num_step_7,num_step_8,num_step_9,
num_step_10,num_step_11,num_step_12,num_step_13,num_step_14,num_step_15,num_step_16,num_step_17,num_step_18)

In [84]:
# video6-25
import torch
num_each_class = [3845, 30, 95, 536, 365 ,862, 293, 896, 1359, 462 ,1304, 389 ,368, 247, 832, 785, 344, 170, 978]
weight = compute_weights_WCE(num_each_class)

tensor([0.0033, 0.4182, 0.1321, 0.0234, 0.0344, 0.0146, 0.0428, 0.0140, 0.0092,
        0.0272, 0.0096, 0.0323, 0.0341, 0.0508, 0.0151, 0.0160, 0.0365, 0.0738,
        0.0128])
